<a href="https://colab.research.google.com/github/jonlwowski012/StockSellRulesChecker/blob/master/StockScreener/Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
! git clone https://github.com/jonlwowski012/StockSellRulesChecker.git
! pip install -r StockSellRulesChecker/StockScreener/requirements.txt

fatal: destination path 'StockSellRulesChecker' already exists and is not an empty directory.


In [7]:
from finviz.screener import Screener
import nest_asyncio
import finviz
import pandas_datareader as pdr
import yfinance as yf
import datetime
import stockquotes
import pandas as pd
import tqdm
nest_asyncio.apply()

In [8]:
eps_5year_over20pct = True
eps_qoq_over20pct = True
eps_yoy_over20pct = True
sales_5years_over20pct = True
sales_qoq_over20pct = True
curr_vol_over200 = True

filters = []
if eps_5year_over20pct:
  filters.append('fa_eps5years_o20')
if eps_qoq_over20pct:
  filters.append('fa_epsqoq_o20')
if eps_yoy_over20pct:
  filters.append('fa_epsyoy_o20')
if sales_5years_over20pct:
  filters.append('fa_sales5years_o20')
if sales_qoq_over20pct:
  filters.append('fa_salesqoq_o20')
if curr_vol_over200:
  filters.append('sh_curvol_o200')

stock_list = Screener(filters=filters, table='Performance', order='price')  

In [ ]:
print(stock_list)

No. | Ticker | Perf Week | Perf Month | Perf Quart | Perf Half | Perf Year | Perf YTD | Volatility W | Volatility M | Recom | Avg Volume | Rel Volume | Price  | Change | Volume 
--- | ------ | --------- | ---------- | ---------- | --------- | --------- | -------- | ------------ | ------------ | ----- | ---------- | ---------- | ------ | ------ | -------
1   | WKHS   | 2.08%     | -53.88%    | -23.37%    | -35.96%   | 1035.37%  | -15.62%  | 11.30%       | 19.52%       | 2.40  | 21.54M     | 0.58       | 16.00  | -4.13% | 772,275
2   | VALE   | 1.06%     | -0.92%     | 2.84%      | 50.12%    | 123.28%   | 3.46%    | 2.48%        | 3.67%        | 2.00  | 29.30M     | 0.28       | 17.17  | -0.09% | 513,988
3   | GNMK   | 19.98%    | 5.10%      | 84.41%     | 90.59%    | 426.43%   | 63.70%   | 9.92%        | 7.98%        | 1.90  | 1.95M      | 6.81       | 23.90  | -0.02% | 824,516
4   | EXPI   | 21.33%    | -28.42%    | 66.32%     | 165.73%   | 1133.41%  | 76.65%   | 11.83%       | 13.20% 

In [11]:
def moving_average(yahoo_df, days):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max()
    end_date = curr_date - datetime.timedelta(days=days)
    
    df_days = df[df['Date'] >= end_date]
    return float(df_days['Close'].mean())

def week52_low_high(yahoo_df):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max()
    end_date = curr_date - datetime.timedelta(days=365)
    
    df_days = df[df['Date'] >= end_date]
    return float(df_days['Close'].max()), float(df_days['Close'].min())


screened_stocks = {}
for stock in tqdm.tqdm(stock_list):
    screened_stocks[stock['Ticker']] = {}
    
    finviz_stats = finviz.get_stock(stock['Ticker'])
    try:
        yahoo_df = pdr.get_data_yahoo(stock['Ticker'], interval = "d")
    except:
        continue
    SMA200_value = moving_average(yahoo_df, days=200)
    SMA150_value = moving_average(yahoo_df, days=150)
    SMA50_value = moving_average(yahoo_df, days=50)
    SMA200_percent = float(finviz_stats['SMA200'].replace("%",""))
    SMA50_percent = float(finviz_stats['SMA50'].replace("%",""))
    inst_own = float(finviz_stats['Inst Own'].replace("%",""))/100
    prev_close = float(finviz_stats['Prev Close'].replace("$",""))
    week52_high, week52_low = week52_low_high(yahoo_df)
    
    screened_stocks[stock['Ticker']]['SMA200_value'] = SMA200_value
    screened_stocks[stock['Ticker']]['SMA150_value'] = SMA150_value
    screened_stocks[stock['Ticker']]['SMA50_value'] = SMA50_value
    screened_stocks[stock['Ticker']]['SMA200_percent'] = SMA200_percent
    screened_stocks[stock['Ticker']]['SMA50_percent'] = SMA50_percent
    screened_stocks[stock['Ticker']]['prev_close'] = prev_close
    screened_stocks[stock['Ticker']]['week52_high'] = week52_high
    screened_stocks[stock['Ticker']]['week52_low'] = week52_low
    screened_stocks[stock['Ticker']]['Inst. Ownership'] = inst_own
    
    # Institutional Ownership < 35%
    if inst_own <= 0.35:
      inst_ownership_rule = True
    else:
      inst_ownership_rule = False
    screened_stocks[stock['Ticker']]['inst_ownership_rule'] = inst_ownership_rule

    # Positive 200d MA positive
    if SMA200_percent > 0:
        SMA200_slope_rule = True
    else:
        SMA200_slope_rule = False
    screened_stocks[stock['Ticker']]['SMA200_slope_rule'] = SMA200_slope_rule
        
    # 150d MA greater than 200d MA
    if SMA150_value > SMA200_value:
        SMA150_greater_SMA200_rule = True
    else:
        SMA150_greater_SMA200_rule = False
    screened_stocks[stock['Ticker']]['SMA150_greater_SMA200_rule'] = SMA150_greater_SMA200_rule
        
    # 50d MA greater than 150d MA
    if SMA50_value > SMA150_value:
        SMA50_greater_SMA150_rule = True
    else:
        SMA50_greater_SMA150_rule = False
    screened_stocks[stock['Ticker']]['SMA50_greater_SMA150_rule'] = SMA50_greater_SMA150_rule
        
    # Close above 50d MA
    if prev_close > SMA50_value:
        close_greater_SMA50_rule = True
    else:
        close_greater_SMA50_rule = False
    screened_stocks[stock['Ticker']]['close_greater_SMA50_rule'] = close_greater_SMA50_rule
        
    # 52 week high low span rule
    if 0.75*week52_high > 1.25*week52_low:
        week52_span_rule = True
    else:
        week52_span_rule = False
    screened_stocks[stock['Ticker']]['week52_span_rule'] = week52_span_rule
    
    # Close above 52 week high - 25%
    if prev_close > 0.75*week52_high:
        close_above_52weekhigh_rule = True
    else:
        close_above_52weekhigh_rule = False
    screened_stocks[stock['Ticker']]['close_above_52weekhigh_rule'] = close_above_52weekhigh_rule

output_list = []
for stock in screened_stocks.keys():
    cols = ["Ticker"] + list(screened_stocks[stock].keys())
    temp_list = []
    temp_list.append(stock)
    for rule in screened_stocks[stock].keys():
        temp_list.append(screened_stocks[stock][rule])
    output_list.append(temp_list)
        
df_out = pd.DataFrame(output_list,columns=cols)

    


100%|██████████| 23/23 [00:10<00:00,  2.22it/s]


In [12]:
df_out

,Ticker,SMA200_value,SMA150_value,SMA50_value,SMA200_percent,SMA50_percent,prev_close,week52_high,week52_low,Inst. Ownership,inst_ownership_rule,SMA200_slope_rule,SMA150_greater_SMA200_rule,SMA50_greater_SMA150_rule,close_greater_SMA50_rule,week52_span_rule,close_above_52weekhigh_rule
0,VTVT,2.085441,2.166863,2.519143,29.04,19.05,2.89,3.190000,1.490000,0.0440,True,True,True,True,True,True,True
1,OPK,4.255147,4.488922,4.986000,12.23,-2.82,4.64,5.990000,1.150000,0.2920,True,True,True,True,False,True,True
2,DVAX,5.836691,6.158333,9.052000,51.88,24.68,9.88,11.700000,2.600000,0.7480,False,True,True,True,True,True,True
3,WKHS,23.663272,23.515245,26.829000,-20.70,-38.53,16.69,41.340000,1.540000,0.3860,False,False,False,True,False,True,False
4,PLYM,14.040735,14.445098,15.578572,21.44,9.65,17.08,17.350000,7.900000,0.6780,False,True,True,True,True,True,True
5,VALE,14.532610,15.684167,17.374714,29.84,-1.48,17.19,18.940001,6.580000,0.2340,True,True,True,True,False,True,True
6,AMRS,6.842794,8.139020,14.851429,243.74,55.01,21.83,22.340000,1.890000,0.3830,False,True,True,True,True,True,True
7,GNMK,14.701029,15.275196,18.310286,61.92,37.35,23.90,23.980000,3.560000,0.9873,False,True,True,True,True,True,True
8,VIPS,25.562831,28.574755,35.844429,78.97,26.49,42.92,42.924999,13.250000,0.6140,False,True,True,True,True,True,True
9,GRWG,33.554118,39.439853,51.070286,96.88,1.03,51.78,64.580002,2.870000,0.5370,False,True,True,True,True,True,True


In [ ]:
df_out.to_csv("screened_stocks.csv")